## Setup

In [ ]:
%pip install -qU writer-sdk

### Initialization

Sign up for [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) and follow this [Quickstart](https://dev.writer.com/api-guides/quickstart) to obtain an API key. Then, set the WRITER_API_KEY environment variable:

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()

### Usage

#### Definition

In [ ]:
if not os.getenv("GRAPH_ID"):
    os.environ["GRAPH_ID"] = getpass.getpass("Enter ID of Graph you want to use as tool: ")

tools = [
    {
        "type": "graph",
        "function": {
            "description": "Prescribing info",
            "graph_ids": [
                os.getenv("GRAPH_ID") #possible to use multiple ids
            ],
            "subqueries": True
        }
    }
]

#### Invocation

In [ ]:
response = client.chat.chat(
    model="palmyra-x-004",
    messages=[{"role": "user", "content": "Which adverse reactions BAVENCIO has?"}],
    tools=tools,
    tool_choice="auto"
)

response.choices[0].message.content

In [ ]:
response.choices[0].message.graph_data

#### Streaming

In [ ]:
response_stream = client.chat.chat(
    model="palmyra-x-004",
    messages=[{"role": "user", "content": "Which adverse reactions BAVENCIO has?"}],
    tools=tools,
    tool_choice="auto",
    stream=True
)

graph_data = None

for chunk in response_stream:
    print(chunk)
    if chunk.choices[0].delta.llm_data is not None:
        llm_data = chunk.choices[0].delta.graph_data

graph_data